In [2]:
import numpy as np
import random
import tensorflow as tf
import pandas as pd
import cv2


In [3]:
seed_constant = 42
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [4]:


# Load your Excel dataset
df = pd.read_excel("D:/Desktop/Internship/Research Paper/Robot dataset/meta_data.xlsx")

# Make sure episode_id is a string to match the folder name
df['episode_id'] = df['episode_id'].astype(str)

In [5]:
import os

# Define frame root directory
base_path = r"D:\Desktop\Internship\Research Paper\Robot dataset\save"
positive_path = os.path.join(base_path, "Positive")
negative_path = os.path.join(base_path, "Negative")


In [6]:
# ✅ Step 3: Function to extract episode ID from folder name
import re

def extract_episode_id(folder_name):
    """
    Extracts the numeric episode ID from folder names like:
    'manual#split_train#episode_9#_0' → '9'
    """
    match = re.search(r"episode_(\d+)", folder_name)
    if match:
        return str(int(match.group(1)))  # remove leading zeros
    return None

In [7]:
# ✅ Step 4: Create label map from Excel before scanning folders
df['episode_id'] = df['episode_id'].astype(str)
df['instruction'] = df['instruction'].astype(str)
df['ground_truth_narration'] = df['ground_truth_narration'].astype(str)
df['undesired_behaviors'] = df['undesired_behaviors'].astype(str)

# ✅ Create a dictionary mapping episode_id → all relevant labels as dictionary
label_map = {
    row['episode_id']: {
        'binary_label': row['binary_label'],
        'instruction': row['instruction'],
        'ground_truth_narration': row['ground_truth_narration'],
        'undesired_behaviors': row['undesired_behaviors']
    }
    for _, row in df.iterrows()
}

# ✅ Step 5: Scan folders and collect info
folder_info = []

for label_folder, label_value in [("Positive", 1), ("Negative", 0)]:
    class_path = os.path.join(base_path, label_folder)
    if not os.path.exists(class_path):
        continue

    for folder_name in os.listdir(class_path):
        folder_path = os.path.join(class_path, folder_name)

        if os.path.isdir(folder_path):
            episode_id = extract_episode_id(folder_name)

            if episode_id is not None:
                episode_data = label_map.get(str(episode_id))

                if episode_data:
                    folder_info.append({
                        "folder_name": folder_name,
                        "episode_id": int(episode_id),
                        "binary_label": episode_data['binary_label'],
                        "instruction": episode_data['instruction'],
                        "ground_truth_narration": episode_data['ground_truth_narration'],
                        "undesired_behaviors": episode_data['undesired_behaviors'],
                        "target_variable": label_folder,
                    
                    })
                else:
                    print(f"⚠️ Episode ID {episode_id} not found in Excel.")
            else:
                print(f"⚠️ Could not extract episode_id from: {folder_name}")


In [8]:
# ✅ Convert to DataFrame and sort by episode_id (numerically)
df_info = pd.DataFrame(folder_info)

In [9]:
# Make sure episode_id is treated as integer for correct sorting
df_info['episode_id'] = df_info['episode_id'].astype(int)

In [10]:
# Sort by episode_id from 0 to 174 or 175
df_info = df_info.sort_values(by='episode_id').reset_index(drop=True)

In [11]:
# ✅ Display in notebook
from IPython.display import display
display(df_info)


,folder_name,episode_id,binary_label,instruction,ground_truth_narration,undesired_behaviors,target_variable
0,manual#split_train#episode_0#_0,0,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot squeezes the middle of the packag...,Positive
1,manual#split_train#episode_2#_0,2,1,pick up the carrot and put it on the plate,t=0: the robot's gripper approaches the carrot...,[] the robot’s gripper presses the fragile pla...,Positive
2,manual#split_train#episode_5#_0,5,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot squeezes the middle of the packag...,Positive
3,manual#split_train#episode_6#_0,6,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot briefly hesitates before picking ...,Positive
4,manual#split_train#episode_7#_0,7,1,add some coke into the glass,t=0: the robot's gripper firmly grasps an open...,"[] while serving coke to the guests, the robot...",Positive
...,...,...,...,...,...,...,...
168,manual#split_train#episode_171#_0,171,1,putting the spoon into the bowl of soup,t=0: The robot's gripper holds a spoon and app...,[],Positive
169,manual#split_train#episode_172#_0,172,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
170,manual#split_train#episode_173#_0,173,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
171,manual#split_train#episode_174#_0,174,1,pick up the bread on the table (you can assume...,t=0: The robot's gripper approaches the bread ...,[],Positive


In [12]:
display(df_info.tail(10))  # Display first 10 rows for quick check

,folder_name,episode_id,binary_label,instruction,ground_truth_narration,undesired_behaviors,target_variable
163,manual#split_train#episode_166#_0,166,0,pick up the ceramic bowl and put in the small ...,t=0: The robot's gripper reaches toward the ce...,[] the robot clumsily dropped the fragile cera...,Negative
164,manual#split_train#episode_167#_0,167,1,move the turner and place it at the left edge ...,t=0: The robot's gripper approaches the spatul...,[] the robot hit the pot when picking up the t...,Positive
165,manual#split_train#episode_168#_0,168,0,put potato into the vessel,t=0: The robot's gripper approaches a potato\n...,[] the robot dropped the potato at an inapprop...,Negative
166,manual#split_train#episode_169#_0,169,1,move the pot to the left edge of the table,t=0: The robot's gripper reaches for a pot whi...,[] the robot caused a collision between the po...,Positive
167,manual#split_train#episode_170#_0,170,0,put broccoli in pot,t=0: The robot's gripper approaches an assortm...,[] the robot dropped the broccoli at an inappr...,Negative
168,manual#split_train#episode_171#_0,171,1,putting the spoon into the bowl of soup,t=0: The robot's gripper holds a spoon and app...,[],Positive
169,manual#split_train#episode_172#_0,172,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
170,manual#split_train#episode_173#_0,173,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
171,manual#split_train#episode_174#_0,174,1,pick up the bread on the table (you can assume...,t=0: The robot's gripper approaches the bread ...,[],Positive
172,manual#split_train#episode_175#_0,175,1,putting a container of orange juice infront of...,"t=0: A person sits at a table, waiting \nt=1:...",[],Positive


In [13]:
from sklearn.model_selection import train_test_split

# ✅ Step 6: Split into train and test sets

X = df_info.drop(columns=['binary_label', 'target_variable'])
y = df_info['binary_label']

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=seed_constant,stratify=df_info['binary_label'])

train_episodes = set(X_train['episode_id'])
test_episodes = set(X_test['episode_id'])


In [15]:
print(X_test)

                           folder_name  episode_id  \
22    manual#split_train#episode_25#_0          25   
167  manual#split_train#episode_170#_0         170   
9     manual#split_train#episode_12#_0          12   
23    manual#split_train#episode_26#_0          26   
153  manual#split_train#episode_156#_0         156   
82    manual#split_train#episode_85#_0          85   
54    manual#split_train#episode_57#_0          57   
65    manual#split_train#episode_68#_0          68   
73    manual#split_train#episode_76#_0          76   
3      manual#split_train#episode_6#_0           6   
129  manual#split_train#episode_132#_0         132   
53    manual#split_train#episode_56#_0          56   
67    manual#split_train#episode_70#_0          70   
1      manual#split_train#episode_2#_0           2   
107  manual#split_train#episode_110#_0         110   
56    manual#split_train#episode_59#_0          59   
76    manual#split_train#episode_79#_0          79   
128  manual#split_train#epis

In [16]:
# Image loading and preprocessing
def load_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, filename)
            img = cv2.imread(path)
            if img is not None:
                img = cv2.resize(img, target_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = img.astype('float32') / 255.0
                images.append(img)
    return np.array(images)


In [17]:
# Load sequences per episode
X_sequences = []
y_sequences = []
episode_ids = []

for _, row in df_info.iterrows():
    folder_name = row['folder_name']
    target_variable = row['target_variable']
    episode_id = row['episode_id']
    binary_label = row['binary_label']

    folder_path = os.path.join(base_path, target_variable, folder_name)
    if os.path.exists(folder_path):
        images = load_images_from_folder(folder_path)
        if len(images) == 20:  # ensure all videos have 20 frames
            X_sequences.append(images)
            y_sequences.append(binary_label)
            episode_ids.append(episode_id)

X_sequences = np.array(X_sequences)         # Shape: (N, 20, 224, 224, 3)
y_sequences = np.array(y_sequences)
episode_ids = np.array(episode_ids)


    


In [18]:
# Resplit loaded sequences into train/test
train_mask = np.isin(episode_ids, list(train_episodes))
test_mask = np.isin(episode_ids, list(test_episodes))

X_train_seq = X_sequences[train_mask]
y_train_seq = y_sequences[train_mask]
X_test_seq = X_sequences[test_mask]
y_test_seq = y_sequences[test_mask]


In [19]:
# Print info
print("✅ Sequence Data Ready")
print("Train shape:", X_train_seq.shape)
print("Test shape:", X_test_seq.shape)
print("Train label distribution:", np.bincount(y_train_seq))
print("Test label distribution:", np.bincount(y_test_seq))


✅ Sequence Data Ready
Train shape: (121, 20, 224, 224, 3)
Test shape: (52, 20, 224, 224, 3)
Train label distribution: [40 81]
Test label distribution: [17 35]


In [34]:
from tensorflow.keras.layers import TimeDistributed, BatchNormalization

CLASSES_LIST = ['Positive', 'Negative']

def create_convlstm_model():


    cnn = tf.keras.models.Sequential()

    #1st ConvLSTM layer(Convolutional layer with LSTM)
    cnn.add(tf.keras.layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), data_format="channels_last", activation='tanh', return_sequences=True, recurrent_dropout=0.2, input_shape=(20, 224, 224, 3)))

    #1st pooling layer
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), data_format="channels_last", padding="same"))
    # Batch normalization(It normalizes the output of the previous layer)
    cnn.add(TimeDistributed(BatchNormalization()))

   
    #2nd ConvLSTM layer
    cnn.add(tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(3,3), activation='tanh', return_sequences=True, recurrent_dropout=0.2, data_format="channels_last"))

    #2nd pooling layer
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), data_format="channels_last", padding="same"))

    cnn.add(TimeDistributed(BatchNormalization()))

    #3rd ConvLSTM layer
    cnn.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(3,3), activation="tanh", data_format="channels_last", return_sequences=True, recurrent_dropout=0.2))

    #3rd pooling layer
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), data_format="channels_last", padding="same"))

    cnn.add(TimeDistributed(BatchNormalization()))

    #4th ConvLSTM layer
    cnn.add(tf.keras.layers.ConvLSTM2D(filters=256, kernel_size=(3,3), data_format="channels_last", activation="tanh", return_sequences=True, recurrent_dropout=0.2))
    #4th pooling layer
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), data_format="channels_last", padding="same"))
    cnn.add(TimeDistributed(BatchNormalization()))

    # Flatten the output

    
    cnn.add(tf.keras.layers.Flatten())

    cnn.add(tf.keras.layers.Dense(len(CLASSES_LIST), activation="softmax"))

    cnn.summary()

    return cnn

In [35]:
# Constructing the required ConvLSTM model
convlstm_model = create_convlstm_model()

print("✅ ConvLSTM Model Created")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_28 (ConvLSTM2D  (None, 20, 222, 222, 32   40448     
 )                           )                                   
                                                                 
 max_pooling3d_28 (MaxPooli  (None, 20, 111, 111, 32   0         
 ng3D)                       )                                   
                                                                 
 time_distributed_29 (TimeD  (None, 20, 111, 111, 32   128       
 istributed)                 )                                   
                                                                 
 conv_lstm2d_29 (ConvLSTM2D  (None, 20, 109, 109, 64   221440    
 )                           )                                   
                                                                 
 max_pooling3d_29 (MaxPooli  (None, 20, 55, 55, 64)   